# Kurvanpassning av en dämpad oscillation
Experimentet utfördes med ett accelerometer som var fäst på en gunga. Gungans (pendelns) amplitud och dess maximala vinkelhastighet avtar exponentiellt med tiden medan pendeln själv följer en nästan perfekt harmonisk oscillation. Vi ska betrakta här vinkelhastigheten av pendeln.

Först som vanligt - några externa bibliotek:

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 9]
import scipy.optimize as opt
import numpy as np
from IPython.display import display, Math, Markdown

Vi ska analysera data som befinner sig i filen "20150224_accgyro_cal.txt" i samma katalog som vår skript.<br/>
Filen innehåller 5942 rader med mätdata i tre kolumner:

1. en tid (i sekunder) för mätningen
1. den uppmätta accelerationen (i $m/s^2$) vid tidpunkten
1. den uppmätta vinkelhastigheten (i $^o/s$)

som vi läser in i tre python-listor `tid`, `acc` och `vel`.<br/> 
Sedan rättar vi till tidsaxeln genom att sätta första tidpunkten till `tid=0`

In [ ]:
tid, acc, vel = np.loadtxt("20150224_accgyro_cal.txt", unpack=True)
tid = tid - min(tid)  # subtraherar min(tid) från alla tidpunkter

Hur ser datan ut då?<br/>
Vi gör en snabb plot:

In [ ]:
fig, ax = plt.subplots()
ax.plot(tid, acc, label="acceleration", color="red", linestyle="", marker="+", markersize=5)
ax.plot(tid, vel, label="vinkelhastighet", color="blue", linestyle="", marker="+", markersize=5)
ax.legend()

Vinkelhastighetens amplitud avtar med tiden. Kan vi göra en kurvanpassning?

Vi kan utgå från följande modellfunktion:<br/>
$f(t) = A\sin\left(2\pi B t + C\right)\cdot e^{-D t}$<br/>
där $A$ är den ursprungliga amplituden vid $t=0$, $B$ är frekvensen, $C$ är en fasförskjutning och $D$ står för den expontiella dämpningen.

In [ ]:
def f(t, A, B, C, D):
    return A*np.sin(2*np.pi*B*t + C)*np.exp(-D*t)

Hur ser denna funktion ut allmänt?

Vi gör en graf för 
$f(t) = 100\sin\left(2\pi t + 0\right)\cdot e^{-0.1 t}$

In [ ]:
guessParams = [100, 1, 0, 0.1]


fig, ax = plt.subplots()
ax.plot(tid, f(tid, *guessParams), label="exponentiell dämpad oscillation", 
        color="red", linestyle="-", marker="", linewidth=2)
ax.plot(tid, vel, label="vinkelhastighet", color="blue", linestyle="", marker="+", markersize=5, alpha=0.1)
ax.legend()

Och nu låter vi Python hitta en kurvanpassning med `opt.curve_fit()`<br/>
Kommer Python att lyckas?

In [ ]:
optimParams, pcov = opt.curve_fit(f=f, xdata=tid, ydata=vel, p0=guessParams, method="trf", verbose=2)

<div class="alert alert-block alert-warning">
Om det inte fungerade, så kan vi testa med andra startvärden för `A`, `B`, `C` och `D`<br/>

Testa följande istället och byt ut<br/>
`guessParams = [100, 1, 0, 0.1]` mot `guessParams = [100, 0.1, 0, 0.1]`<br/>
i koden.
</div>

In [ ]:
print("anpassad: A={:8g}, B={:8g}, C={:8g}".format(*optimParams))

display(Markdown(r"$f(t) = {:8g} \sin\left( 2\pi\cdot{:+8g} t {:+8g}^o \right)$".format(*optimParams)))

In [ ]:
fig, ax = plt.subplots()
ax.plot(tid, vel, label="vinkelhastighet", color="blue", linestyle="", marker="+", markersize=5)
ax.plot(tid, f(tid, *optimParams), "r-", label = "anpassad", linewidth=2)
ax.plot(tid, f(tid, *guessParams), "c-", label = "vår gisnning", linewidth=1, alpha=0.5)
ax.legend()
ax.set_xlabel(r"tid $[s]$")
ax.set_ylabel(r"vinkelhastighet $[^o/s]$")